# LSTM and Neural Network Approach to Target Ratio Prediction
-Team Abraca_dataIITKGP.
Team members:
Nikhil Mantri, Tanmay Das, Aniket Shah, Devansh Pandey

## We designed a time series data combined with Neural Network approach to Target ratio Prediction
First of all we needed to arrange the data in feature_00 till feature_60 of every type(ER1, ER2, ER3, MI1, MI2) in an order of time series. To do this, we hypothesised that the dependence of target ratio will be higher on yesterday than the days before yesterday and extrapolated the logic to the n days(n=60 here) before it. So highest covraince day is the 60th day, next highest 59th and so on.

We then took 1-59 days as our train data vectors in sequence row-wise and 60th day as the target into a custom LSTM network defined as regressor in the following cells. So each type has its regressor. And these regressors will be seperately defined and trained on the train and test dataset. Next create feature_ratio_type(0 till 5) by outpitting the corresponding prediction of a type on "(60+span[i] day)/60th day" by using our LSTM trained for each type on each of the train and test dataset.
This we are terming as feature ratios. 

Later we put these six ultimate features in a custom dense neural network trained on these 6 train feature ratios and target ratio as the target variable for 100000 epochs, batch_size=16. We standard scale the data before putting it in our neural network and inverse tranform the output that is fit for submission .
We achieved a stagarring leaderboard score of (0.984214) which was e raised to -ve of RMSE value on test dataset.

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
from keras.models import load_model
# import warnings 
df_original=pd.read_csv("weights/train.csv")
train = pd.read_csv("weights/train.csv')
test=pd.read_csv("weights/test.csv")

Using TensorFlow backend.


In [2]:
t = ['id','span','target']
for i in range(0, 6):
#     pd.concat([train.iloc[:,2:3],train.iloc[:,65:67]], axis=1)
    a = np.array(pd.concat([train.iloc[:,2:3], train.iloc[:,(3+i*60):(3+(i+1)*60)]], axis=1).cov()['target'].drop(['target']).sort_values().index)
    t = np.append(t, values=a)
train_sequence = train[t]
test_sequence = test[np.delete(t, 2)]

In [3]:
for col in  train_sequence.columns:
    train_sequence[col] = pd.to_numeric(train_sequence[col], errors='coerce')

    
for col in  test_sequence.columns:
    test_sequence[col] = pd.to_numeric(test_sequence[col], errors='coerce')

In [4]:
#filling missing values
for col in train_sequence.columns:
    if train_sequence[col].isnull().sum()>0:
        train_sequence[col].fillna(train_sequence[col].mean,inplace=True)
        
#filling missing values
for col in test_sequence.columns:
    if test_sequence[col].isnull().sum()>0:
        test_sequence[col].fillna(test_sequence[col].mean,inplace=True)

In [5]:
train_sequence_transpose = train_sequence.T
train_sequence_transpose.columns

test_sequence_transpose = test_sequence.T
test_sequence_transpose.columns

RangeIndex(start=0, stop=1000, step=1)

In [ ]:
# #
# #For train dataset

# type_00 = []
# type_01 = []
# type_02 = []
# type_03 = []
# type_04 = []
# type_05 = []
# for i in range(test_sequence.shape[0]):
#     type_00.append(train_sequence_transpose[i][2:61])
#     type_01.append(train_sequence_transpose[i][62:121])
#     type_02.append(train_sequence_transpose[i][122:181])
#     type_03.append(train_sequence_transpose[i][182:241])
#     type_04.append(test_sequence_transpose[i][242:301])
#     type_05.append(train_sequence_transpose[i][302:361])
    
# import numpy as np
# type_00 = np.array(type_00)
# type_01 = np.array(type_01)
# type_02 = np.array(type_02)
# type_03 = np.array(type_03)
# type_04 = np.array(type_04)
# type_05 = np.array(type_05)
# type_00 = type_00.reshape((train_sequence.shape[0], 59))
# type_01 = type_01.reshape((train_sequence.shape[0], 59))
# type_02 = type_02.reshape((train_sequence.shape[0], 59))
# type_03 = type_03.reshape((train_sequence.shape[0], 59))
# type_04 = type_04.reshape((train_sequence.shape[0], 59))
# type_05 = type_05.reshape((train_sequence.shape[0], 59))

# type_00_target = []
# type_01_target = []
# type_02_target = []
# type_03_target = []
# type_04_target = []
# type_05_target = []
# for i in range(test_sequence.shape[0]):
#     type_00_target.append(test_sequence_transpose[i][61])
#     type_01_target.append(test_sequence_transpose[i][121])
#     type_02_target.append(test_sequence_transpose[i][181])
#     type_03_target.append(test_sequence_transpose[i][241])
#     type_04_target.append(test_sequence_transpose[i][301])
#     type_05_target.append(test_sequence_transpose[i][361])

In [ ]:
#
#For test dataset

type_00 = []
type_01 = []
type_02 = []
type_03 = []
type_04 = []
type_05 = []
for i in range(test_sequence.shape[0]):
    type_00.append(test_sequence_transpose[i][2:61])
    type_01.append(test_sequence_transpose[i][62:121])
    type_02.append(test_sequence_transpose[i][122:181])
    type_03.append(test_sequence_transpose[i][182:241])
    type_04.append(test_sequence_transpose[i][242:301])
    type_05.append(test_sequence_transpose[i][302:361])
    
import numpy as np
type_00 = np.array(type_00)
type_01 = np.array(type_01)
type_02 = np.array(type_02)
type_03 = np.array(type_03)
type_04 = np.array(type_04)
type_05 = np.array(type_05)
type_00 = type_00.reshape((test_sequence.shape[0], 59))
type_01 = type_01.reshape((test_sequence.shape[0], 59))
type_02 = type_02.reshape((test_sequence.shape[0], 59))
type_03 = type_03.reshape((test_sequence.shape[0], 59))
type_04 = type_04.reshape((test_sequence.shape[0], 59))
type_05 = type_05.reshape((test_sequence.shape[0], 59))

type_00_target = []
type_01_target = []
type_02_target = []
type_03_target = []
type_04_target = []
type_05_target = []
for i in range(test_sequence.shape[0]):
    type_00_target.append(test_sequence_transpose[i][61])
    type_01_target.append(test_sequence_transpose[i][121])
    type_02_target.append(test_sequence_transpose[i][181])
    type_03_target.append(test_sequence_transpose[i][241])
    type_04_target.append(test_sequence_transpose[i][301])
    type_05_target.append(test_sequence_transpose[i][361])

In [ ]:
import numpy as np
type_00 = np.reshape(type_00, (type_00.shape[0], type_00.shape[1], 1))
type_01 = np.reshape(type_01, (type_01.shape[0], type_01.shape[1], 1))
type_02 = np.reshape(type_02, (type_02.shape[0], type_02.shape[1], 1))
type_03 = np.reshape(type_03, (type_03.shape[0], type_03.shape[1], 1))
type_04 = np.reshape(type_04, (type_04.shape[0], type_04.shape[1], 1))
type_05 = np.reshape(type_05, (type_05.shape[0], type_05.shape[1], 1))

In [ ]:
for j in range(len(type_00[0])):      
    for i in range(len(type_00)):
        for k in range(len(type_00[0][0])):            
            if type(type_00[0][0][0])!=type(type_00[i][j][k]):
                type_00[i][j][k]=0
                                    
for j in range(len(type_01[0])):      
    for i in range(len(type_01)):
        for k in range(len(type_01[0][0])):            
            if type(type_01[0][0][0])!=type(type_01[i][j][k]):
                type_01[i][j][k]=0
                                        
for j in range(len(type_02[0])):      
    for i in range(len(type_02)):
        for k in range(len(type_02[0][0])):            
            if type(type_02[0][0][0])!=type(type_02[i][j][k]):
                type_02[i][j][k]=0

for j in range(len(type_03[0])):      
    for i in range(len(type_03)):
        for k in range(len(type_03[0][0])):            
            if type(type_03[0][0][0])!=type(type_03[i][j][k]):
                type_03[i][j][k]=0
                                            
for j in range(len(type_04[0])):      
    for i in range(len(type_04)):
        for k in range(len(type_04[0][0])):            
            if type(type_04[0][0][0])!=type(type_04[i][j][k]):
                type_04[i][j][k]=0

for j in range(len(type_05[0])):      
    for i in range(len(type_05)):
        for k in range(len(type_05[0][0])):            
            if type(type_05[0][0][0])!=type(type_05[i][j][k]):
                type_05[i][j][k]=0

###TARGETS###                
for i in range(len(type_00_target)):    
    if type(type_00_target[0])!=type(type_00_target[i]):        
        type_00_target[i]=1

for i in range(len(type_01_target)):    
    if type(type_01_target[0])!=type(type_01_target[i]):        
        type_01_target[i]=1
        
for i in range(len(type_02_target)):    
    if type(type_02_target[0])!=type(type_02_target[i]):        
        type_02_target[i]=1
        
for i in range(len(type_03_target)):    
    if type(type_03_target[0])!=type(type_03_target[i]):        
        type_03_target[i]=1
        
for i in range(len(type_04_target)):    
    if type(type_04_target[0])!=type(type_04_target[i]):        
        type_04_target[i]=1

for i in range(len(type_05_target)):    
    if type(type_05_target[0])!=type(type_05_target[i]):        
        type_05_target[i]=1

In [ ]:
# regressor = Sequential()

# # Adding the first LSTM layer and some Dropout regularisation
# regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (59, 1)))
# regressor.add(Dropout(0.2))
# regressor.add(LSTM(units = 50, return_sequences = True))
# regressor.add(Dropout(0.2))
# regressor.add(LSTM(units = 50, return_sequences = True))
# regressor.add(Dropout(0.2))
# regressor.add(LSTM(units = 50))
# regressor.add(Dropout(0.2))
# regressor.add(Dense(units = 1))
# # Compiling the RNN
# regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
# # Fitting the RNN to the Training set
# regressor.fit(type_00, type_00_target, epochs = 20, batch_size = 32)

In [ ]:
""""THIS IS FOR THE TRAIN'S FEATURES HENCE HASHED OUT """
# import pickle
# # pickle.dump(regressor,open("weights/regressor00.sav","wb"))
# regressor00=pickle.load(open("weights/regressor00.sav","rb"))
# # pickle.dump(regressor,open("weights/regressor00.sav","wb"))
# regressor01=pickle.load(open("weights/regressor01.sav","rb"))
# # pickle.dump(regressor,open("weights/regressor00.sav","wb"))
# regressor02=pickle.load(open("weights/regressor02.sav","rb"))
# # pickle.dump(regressor,open("weights/regressor00.sav","wb"))
# regressor03=pickle.load(open("weights/regressor03.sav","rb"))
# # pickle.dump(regressor,open("weights/regressor00.sav","wb"))
# regressor04=pickle.load(open("weights/regressor04.sav","rb"))
# # pickle.dump(regressor,open("weights/regressor00.sav","wb"))
# regressor05=pickle.load(open("weights/regressor05.sav","rb"))

In [ ]:
import pickle
# pickle.dump(regressor,open("regressor00.sav","wb"))
regressor00=pickle.load(open("weights/regressor00_test.sav","rb"))
# pickle.dump(regressor,open("regressor00.sav","wb"))
regressor01=pickle.load(open("weights/regressor01_test.sav","rb"))
# pickle.dump(regressor,open("regressor00.sav","wb"))
regressor02=pickle.load(open("weights/regressor02_test.sav","rb"))
# pickle.dump(regressor,open("regressor00.sav","wb"))
regressor03=pickle.load(open("weights/regressor03_test.sav","rb"))
# pickle.dump(regressor,open("regressor00.sav","wb"))
regressor04=pickle.load(open("weights/regressor04_test.sav","rb"))
# pickle.dump(regressor,open("regressor00.sav","wb"))
regressor05=pickle.load(open("weights/regressor05_test.sav","rb"))

In [ ]:
""""THIS IS FOR THE TRAIN'S FEATURES HENCE HASHED OUT """
# type_00_copy=type_00
# ans00=[]
# for i in range(len(train_sequence["span"])):
#     span_value=train_sequence["span"][i]
#     for j in range(span_value+1):
#         z=type_00_copy[i][0:59].reshape(1,59,1)
#         temp=regressor00.predict(z)
#         if(j==0):
#             denominator=temp[0][0]
#         for k in range(0,58):
#             type_00_copy[i][k]=type_00_copy[i][k+1]
#         type_00_copy[i][58]=temp
#     if(i%100==0):
#         print(i,"done")
#     ans00.append(temp[0][0]/denominator)
# print("done all")
# print("ans00:",ans00)

# type_01_copy=type_01
# ans01=[]
# for i in range(len(train_sequence["span"])):
#     span_value=train_sequence["span"][i]
#     for j in range(span_value+1):
#         z=type_01_copy[i][0:59].reshape(1,59,1)
#         temp=regressor01.predict(z)
#         if(j==0):
#             denominator=temp[0][0]
#         for k in range(0,58):
#             type_01_copy[i][k]=type_01_copy[i][k+1]
#         type_01_copy[i][58]=temp
#     if(i%100==0):
#         print(i,"done")
#     ans01.append(temp[0][0]/denominator)
# print("done all")
# print("ans01:",ans01)

# type_02_copy=type_02
# ans02=[]
# for i in range(len(train_sequence["span"])):
#     span_value=train_sequence["span"][i]
#     for j in range(span_value+1):
#         z=type_02_copy[i][0:59].reshape(1,59,1)
#         temp=regressor02.predict(z)
#         if(j==0):
#             denominator=temp[0][0]
#         for k in range(0,58):
#             type_02_copy[i][k]=type_02_copy[i][k+1]
#         type_02_copy[i][58]=temp
#     if(i%100==0):
#         print(i,"done")
#     ans02.append(temp[0][0]/denominator)
# print("done all")
# print("ans02:",ans02)

# type_03_copy=type_03
# ans03=[]
# for i in range(len(train_sequence["span"])):
#     span_value=train_sequence["span"][i]
#     for j in range(span_value+1):
#         z=type_03_copy[i][0:59].reshape(1,59,1)
#         temp=regressor03.predict(z)
#         if(j==0):
#             denominator=temp[0][0]
#         for k in range(0,58):
#             type_03_copy[i][k]=type_03_copy[i][k+1]
#         type_03_copy[i][58]=temp
#     if(i%100==0):
#         print(i,"done")
#     ans03.append(temp[0][0]/denominator)
# print("done all")
# print("ans03:",ans03)

# type_04_copy=type_04
# ans04=[]
# for i in range(len(train_sequence["span"])):
#     span_value=train_sequence["span"][i]
#     for j in range(span_value+1):
#         z=type_04_copy[i][0:59].reshape(1,59,1)
#         temp=regressor04.predict(z)
#         if(j==0):
#             denominator=temp[0][0]
#         for k in range(0,58):
#             type_04_copy[i][k]=type_04_copy[i][k+1]
#         type_04_copy[i][58]=temp
#     if(i%100==0):
#         print(i,"done")
#     ans04.append(temp[0][0]/denominator)
# print("done all")
# print("ans04:",ans04)

# type_05_copy=type_05
# ans05=[]
# for i in range(len(train_sequence["span"])):
#     span_value=train_sequence["span"][i]
#     for j in range(span_value+1):
#         z=type_05_copy[i][0:59].reshape(1,59,1)
#         temp=regressor05.predict(z)
#         if(j==0):
#             denominator=temp[0][0]
#         for k in range(0,58):
#             type_05_copy[i][k]=type_05_copy[i][k+1]
#         type_05_copy[i][58]=temp
#     if(i%100==0):
#         print(i,"done")
#     ans05.append(temp[0][0]/denominator)
# print("done all")
# print("ans05:",ans05)

# t=pd.DataFrame()
# t['id']=test['id']
# t['span']=test['span']
# t['feature_ratio_type0']=ans00
# t['feature_ratio_type1']=ans01
# t['feature_ratio_type2']=ans02
# t['feature_ratio_type3']=ans03
# t['feature_ratio_type4']=ans04
# t['feature_ratio_type5']=ans05
# t.to_csv("competition_data/LSTM_train_feat_ratios.csv",index=False)


# df_LSTM_train=t

In [ ]:
# #
# regressor00.fit(type_00, type_00_target, epochs = 20, batch_size = 32)
# regressor01.fit(type_01, type_01_target, epochs = 20, batch_size = 32)
# regressor02.fit(type_02, type_02_target, epochs = 20, batch_size = 32)
# regressor03.fit(type_03, type_03_target, epochs = 20, batch_size = 32)
# regressor04.fit(type_04, type_04_target, epochs = 20, batch_size = 32)
# regressor05.fit(type_05, type_05_target, epochs = 20, batch_size = 32)

In [ ]:
type_00_copy=type_00
ans00=[]
for i in range(len(test_sequence["span"])):
    span_value=test_sequence["span"][i]
    for j in range(span_value+1):
        z=type_00_copy[i][0:59].reshape(1,59,1)
        temp=regressor00.predict(z)
        if(j==0):
            denominator=temp[0][0]
        for k in range(0,58):
            type_00_copy[i][k]=type_00_copy[i][k+1]
        type_00_copy[i][58]=temp
    if(i%100==0):
        print(i,"done")
    ans00.append(temp[0][0]/denominator)
print("done all")
print("ans00:",ans00)

In [ ]:
type_01_copy=type_01
ans01=[]
for i in range(len(test_sequence["span"])):
    span_value=test_sequence["span"][i]
    for j in range(span_value+1):
        z=type_01_copy[i][0:59].reshape(1,59,1)
        temp=regressor01.predict(z)
        if(j==0):
            denominator=temp[0][0]
        for k in range(0,58):
            type_01_copy[i][k]=type_01_copy[i][k+1]
        type_01_copy[i][58]=temp
    if(i%100==0):
        print(i,"done")
    ans01.append(temp[0][0]/denominator)
print("done all")
print("ans01:",ans01)

In [ ]:
type_02_copy=type_02
ans02=[]
for i in range(len(test_sequence["span"])):
    span_value=test_sequence["span"][i]
    for j in range(span_value+1):
        z=type_02_copy[i][0:59].reshape(1,59,1)
        temp=regressor02.predict(z)
        if(j==0):
            denominator=temp[0][0]
        for k in range(0,58):
            type_02_copy[i][k]=type_02_copy[i][k+1]
        type_02_copy[i][58]=temp
    if(i%100==0):
        print(i,"done")
    ans02.append(temp[0][0]/denominator)
print("done all")
print("ans02:",ans02)

In [ ]:
type_03_copy=type_03
ans03=[]
for i in range(len(test_sequence["span"])):
    span_value=test_sequence["span"][i]
    for j in range(span_value+1):
        z=type_03_copy[i][0:59].reshape(1,59,1)
        temp=regressor03.predict(z)
        if(j==0):
            denominator=temp[0][0]
        for k in range(0,58):
            type_03_copy[i][k]=type_03_copy[i][k+1]
        type_03_copy[i][58]=temp
    if(i%100==0):
        print(i,"done")
    ans03.append(temp[0][0]/denominator)
print("done all")
print("ans03:",ans03)

In [ ]:
type_04_copy=type_04
ans04=[]
for i in range(len(test_sequence["span"])):
    span_value=test_sequence["span"][i]
    for j in range(span_value+1):
        z=type_04_copy[i][0:59].reshape(1,59,1)
        temp=regressor04.predict(z)
        if(j==0):
            denominator=temp[0][0]
        for k in range(0,58):
            type_04_copy[i][k]=type_04_copy[i][k+1]
        type_04_copy[i][58]=temp
    if(i%100==0):
        print(i,"done")
    ans04.append(temp[0][0]/denominator)
print("done all")
print("ans04:",ans04)

In [ ]:
type_05_copy=type_05
ans05=[]
for i in range(len(test_sequence["span"])):
    span_value=test_sequence["span"][i]
    for j in range(span_value+1):
        z=type_05_copy[i][0:59].reshape(1,59,1)
        temp=regressor05.predict(z)
        if(j==0):
            denominator=temp[0][0]
        for k in range(0,58):
            type_05_copy[i][k]=type_05_copy[i][k+1]
        type_05_copy[i][58]=temp
    if(i%100==0):
        print(i,"done")
    ans05.append(temp[0][0]/denominator)
print("done all")
print("ans05:",ans05)

In [ ]:
t=pd.DataFrame()
t['id']=test['id']
t['span']=test['span']
t['feature_ratio_type0']=ans00
t['feature_ratio_type1']=ans01
t['feature_ratio_type2']=ans02
t['feature_ratio_type3']=ans03
t['feature_ratio_type4']=ans04
t['feature_ratio_type5']=ans05
t.to_csv("competition_data/LSTM_test_feat_ratios.csv",index=False)

In [ ]:
#df_LSTM=pd.read_csv("competition_data/LSTM_test_feat_ratios.csv")
df_LSTM_test=t

In [ ]:
df_original.drop("id",axis=1,inplace=True)

In [ ]:
for col in df_LSTM_train.columns:
    if df_LSTM_train[col].isnull().sum()>0:
        df_LSTM_train[col].fillna(df_LSTM_train[col].mean,inplace=True)

for col in df_LSTM_test.columns:
    if df_LSTM_test[col].isnull().sum()>0:
        df_LSTM_test[col].fillna(df_LSTM_test[col].mean,inplace=True)

In [ ]:
xtrain=df_LSTM_train.iloc[:].values
ytrain=df_original['target'].values

In [ ]:
test_x=df_LSTM_test.iloc[:].values

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
xtrain=sc_x.fit_transform(xtrain)
test_x=sc_x.transform(test_x)
sc_y=StandardScaler()
ytrain_scaled=sc_y.fit_transform(ytrain)

 wE NOW DEFINE OUR NEURAL NET TRAINED ON THE 6 FEATURE RATIOS OF TRAIN AND OUPUT PREDICTED ON TEST DATASET'S FEATURES

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(256, kernel_initializer='normal',input_dim = x.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(16, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(8, kernel_initializer='normal',activation='relu'))
# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
NN_model.fit(xtrain, ytrain_scaled, epochs=1000000, batch_size=16)

In [ ]:
NN_model=load_model("weights/final_NN_over_LSTM.h5")

In [ ]:
ypred=sc_y.inverse_transform(NN_model.predict(test_x))

In [ ]:
finalcsv=pd.DataFrame()
finalcsv['id']=df_LSTM_safe['id']
finalcsv['target']=ypred
finalcsv.to_csv("competition_data/LSTM_plus_NN1.csv",index=False)

pLEASE SUBMIT THE CSV THAT WILL BE CREATED ABOVE AND VERIFY